In [1]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
#---

# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf
using SARSOP
using JLD


(process:5946): Gtk-WARNING **: 14:28:56.240: Locale not supported by C library.
	Using the fallback 'C' locale.


In [2]:
num_x_pts = 50 # e.g. 50
num_y_pts = 50 # e.g. 50
num_th_pts = 20 # e.g. 20
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts);

In [3]:
vlist = [0, 0.5, 1, 2]
omlist = [-1, -0.5, -0.1, 0, 0.1, 0.5, 1]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist));

In [4]:
sensor = Bumper();

In [5]:
config = 3 # 1,2, or 3
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace, aspace=aspace));

In [6]:
num_particles = 2000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

In [7]:
solver = SARSOPSolver()
p = solve(solver, m);

Generating a pomdpx file: model.pomdpx


MethodError: MethodError: no method matching pdf(::AA228FinalProject.RoombaInitialDistribution{RoombaPOMDP{Bumper,Bool}}, ::RoombaState)
Closest candidates are:
  pdf(!Matched::Distributions.Distribution{Distributions.Multivariate,S} where S<:Distributions.ValueSupport, ::AbstractArray{T,1} where T) at /Users/shengji/.julia/packages/Distributions/WHjOk/src/multivariates.jl:179
  pdf(!Matched::Distributions.Distribution{Distributions.Univariate,S} where S<:Distributions.ValueSupport, ::AbstractArray) at deprecated.jl:53
  pdf(!Matched::POMDPModelTools.SparseCat{V<:AbstractArray,P<:AbstractArray}, ::Any) where {V<:AbstractArray, P<:AbstractArray} at /Users/shengji/.julia/packages/POMDPModelTools/eHEjm/src/distributions/sparse_cat.jl:47
  ...

In [9]:
# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, p, belief_updater, max_steps=100))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.1) # to slow down the simulation
end

In [15]:
using JLD
JLD.save("FIB_Bumper_policy.jld", "policy", p)

In [10]:
dist = initialstate_distribution(m)
b0 = initialize_belief(belief_updater, dist)

ParticleCollection{RoombaState}(RoombaState[[11.8367, 2.65306, 1.81882, 0.0], [-17.7551, -10.6122, 3.14159, 0.0], [-2.44898, 0.612245, -0.826735, 0.0], [5.71429, 3.67347, 0.165347, 0.0], [-17.7551, -12.6531, -2.8109, 0.0], [12.8571, -1.42857, 0.496041, 0.0], [12.8571, 2.65306, 0.826735, 0.0], [-14.6939, -4.4898, -2.4802, 0.0], [-9.59184, -3.46939, 2.14951, 0.0], [-6.53061, -1.42857, -0.165347, 0.0]  …  [-10.6122, -3.46939, 0.826735, 0.0], [-4.4898, -3.46939, -2.14951, 0.0], [-6.53061, -0.408163, 1.15743, 0.0], [12.8571, -2.44898, -1.81882, 0.0], [7.7551, -3.46939, 3.14159, 0.0], [-17.7551, -8.57143, 0.496041, 0.0], [-14.6939, -1.42857, -2.8109, 0.0], [1.63265, 1.63265, -0.496041, 0.0], [-22.8571, -2.44898, 0.826735, 0.0], [-22.8571, -9.59184, 0.496041, 0.0]], nothing)

In [13]:
typeof(b0)

ParticleCollection{RoombaState}

In [13]:
using Distributions

In [15]:
Distributions.MultivariateDistribution

Distribution{Multivariate,S} where S<:ValueSupport

In [17]:
dist = initialstate_distribution(m)
b0 = Multinomial(, k)

UndefVarError: UndefVarError: MvNormall not defined

In [15]:
Distributions

Distributions

In [16]:
struct aaa <: Multinomial
end

ErrorException: invalid subtyping in definition of aaa